In [382]:
# You may want to install "gprof2dot"
import io
from collections import Counter

import numpy as np
import scipy.io
import sklearn.model_selection
import sklearn.tree
from numpy import genfromtxt
from scipy import stats
from sklearn.base import BaseEstimator, ClassifierMixin
import math

import pydot

eps = 1e-5  # a small number

In [383]:
#4.1 Decision Trees

class DecisionTree:
    def __init__(self, max_depth=3, feature_labels=None):
        self.max_depth = max_depth
        self.features = feature_labels
        self.left, self.right = None, None  # for non-leaf nodes
        self.split_idx, self.thresh = None, None  # for non-leaf nodes
        self.data, self.pred = None, None  # for leaf nodes

    @staticmethod
    # A method that takes in some feature of the data, the labels and a threshold, and compute 
    # the information gain of a split using the threshold.
    def information_gain(X, y, thresh):
        # A method that takes in the labels of data stored at a node and compute the entropy 
        def gini_impurity(X, y):  
            d = dict()
            for i in range(len(X)):
                if y[i] in d:
                    d[y[i]] += 1
                else:
                    d[y[i]] = 1

            entropy = 0
            for key in d:
                p_c = d[key]/len(X)
                entropy -= p_c * math.log2(p_c)
            return entropy
    
        # Compute current entropy
        H_S = gini_impurity(X, y)
        
        # Compute entropy after splitting by threshold.
        left_X, left_y = [], []
        right_X, right_y = [], []
        for i in range(len(X)):
            if X[i] < thresh:
                left_X.append(X[i])
                left_y.append(y[i])
            else:
                right_X.append(X[i])
                right_y.append(y[i])
                
        # Compute information gain        
        H_l = gini_impurity(left_X, left_y)
        H_r = gini_impurity(right_X, right_y)
        H_after = (H_l * len(left_X) + H_r * len(right_X)) / len(X)
        return H_S - H_after

    def split(self, X, y, idx, thresh):
        X0, idx0, X1, idx1 = self.split_test(X, idx=idx, thresh=thresh)
        y0, y1 = y[idx0], y[idx1]
        return X0, y0, X1, y1

    def split_test(self, X, idx, thresh):
        idx0 = np.where(X[:, idx] < thresh)[0]
        idx1 = np.where(X[:, idx] >= thresh)[0]
        X0, X1 = X[idx0, :], X[idx1, :]
        return X0, idx0, X1, idx1

    # Grows a decision tree by constructing nodes. Using the entropy and segmenter methods, it attempts 
    # to find a configuration of nodes that best splits the input data. This function figures out the split 
    # rules that each node should have and figures out when to stop growing the tree and insert a leaf node. 
    # There are many ways to implement this, but eventually your DecisionTree should store the root node 
    # of the resulting tree so you can use the tree for classification later on. Since the height of your 
    # DecisionTree shouldn’t be astronomically large (you may want to cap the height—if you do, the max height 
    # would be a hyperparameter), this method is best implemented recursively.
    
    def fit(self, X, y):
        if self.max_depth > 0:
            # compute entropy gain for all single-dimension splits,
            # thresholding with a linear interpolation of 10 values
            gains = []
            # The following logic prevents thresholding on exactly the minimum
            # or maximum values, which may not lead to any meaningful node splits.
            
            thresh = np.array([np.linspace(np.min(X[:, i]) + eps, np.max(X[:, i]) - eps, num=10) for i in range(X.shape[1])])
            for i in range(X.shape[1]):
                gains.append([self.information_gain(X[:, i], y, t) for t in thresh[i, :]])
                
            gains = np.nan_to_num(np.array(gains))
            self.split_idx, thresh_idx = np.unravel_index(np.argmax(gains), gains.shape)
            self.thresh = thresh[self.split_idx, thresh_idx]
            X0, y0, X1, y1 = self.split(X, y, idx=self.split_idx, thresh=self.thresh)
            
            if X0.size > 0 and X1.size > 0:
                self.left = DecisionTree(max_depth=self.max_depth - 1, feature_labels=self.features)
                self.left.fit(X0, y0)
                self.right = DecisionTree(max_depth=self.max_depth - 1, feature_labels=self.features)
                self.right.fit(X1, y1)
            else:
                self.max_depth = 0
                self.data, self.labels = X, y
                self.pred = stats.mode(y).mode[0]
        else:
            self.data, self.labels = X, y
            self.pred = stats.mode(y).mode[0]
        return self

    # Given a data point, traverse the tree to find the best label to classify the data point as. Start at 
    # the root node you stored and evaluate split rules at each node as you traverse until you reach a leaf node, 
    # then choose that leaf node’s label as your output label.
    def predict(self, X):
        if self.max_depth == 0:
            return self.pred * np.ones(X.shape[0])
        else:
            X0, idx0, X1, idx1 = self.split_test(X, idx=self.split_idx, thresh=self.thresh)
            yhat = np.zeros(X.shape[0])
            yhat[idx0] = self.left.predict(X0)
            yhat[idx1] = self.right.predict(X1)
            return yhat

    def __repr__(self):
        if self.max_depth == 0:
            return "%s (%s)" % (self.pred, self.labels.size)
        else:
            return "[%s < %s: %s | %s]" % (self.features[self.split_idx],
                                           self.thresh, self.left.__repr__(),
                                           self.right.__repr__())
    

In [384]:
#4.2 Random Forests

class RandomForest:
    def __init__(self, random_states, max_depth, feature_labels):
        self.random_states = random_states
        self.max_depth = max_depth
        self.feature_labels = feature_labels
        
    def fit_and_predict(self, training_X, training_y, test_X):
        predictions = [0] * len(test_X)
        for i in range(self.random_states):
            dt_titanic = DecisionTree(max_depth=3, feature_labels=features)
            rand_X = []
            rand_y = []
            for i in range(len(training_X)):
                randIndex = np.random.randint(len(training_X))
                rand_X.append(training_X[randIndex])
                rand_y.append(training_y[randIndex])
            dt = DecisionTree(self.max_depth, self.feature_labels)
            dt.fit(np.array(rand_X), np.array(rand_y))
            predictions += dt.predict(test_X)
        predictions *= 1/self.random_states
        return np.around(predictions)      

In [385]:
def preprocess(data, fill_mode=True, min_freq=10, onehot_cols=[]):
    # fill_mode = False

    # Temporarily assign -1 to missing data
    data[data == ''] = '-1'

    # Hash the columns (used for handling strings)
    onehot_encoding = []
    onehot_features = []
    for col in onehot_cols:
        counter = Counter(data[:, col])
        for term in counter.most_common():
            if term[0] == '-1':
                continue
            if term[-1] <= min_freq:
                break
            onehot_features.append(term[0])
            onehot_encoding.append((data[:, col] == term[0]).astype(float))
        data[:, col] = '0'
    onehot_encoding = np.array(onehot_encoding).T
    data = np.hstack([np.array(data, dtype=float), np.array(onehot_encoding)])

    # Replace missing data with the mode value. We use the mode instead of
    # the mean or median because this makes more sense for categorical
    # features such as gender or cabin type, which are not ordered.
    if fill_mode:
        for i in range(data.shape[-1]):
            mode = stats.mode(data[((data[:, i] < -1 - eps) +
                                    (data[:, i] > -1 + eps))][:, i]).mode[0]
            data[(data[:, i] > -1 - eps) * (data[:, i] < -1 + eps)][:, i] = mode

    return data, onehot_features

In [386]:
def evaluate(clf):
    print("Cross validation", sklearn.model_selection.cross_val_score(clf, X, y))
    if hasattr(clf, "decision_trees"):
        counter = Counter([t.tree_.feature[0] for t in clf.decision_trees])
        first_splits = [(features[term[0]], term[1]) for term in counter.most_common()]
        print("First splits", first_splits)

In [387]:
# Calculates accuracy given predictions and labels.
def accuracy(X, y):
    total_correct = 0
    for i in range(len(X)):
        if X[i] == y[i]:
            total_correct += 1
    return total_correct/len(X)

In [388]:
# Returns indices for training_labels and training_data that are set aside for validation and for training. Returns
# validation and training indices.
def shuffle_partition(data, count, total):
    np.random.seed(189)
    indices = np.arange(0, total, 1)
    np.random.shuffle(indices)
    return indices[0:count], indices[count:]

In [389]:
# Titanic Dataset

dataset = "titanic"
params = {
    "max_depth": 5,
    #"random_state": 6,
    "min_samples_leaf": 10,
}
N = 100

# Load titanic data
path_train = './dataset/titanic/titanic_training.csv'
data = genfromtxt(path_train, delimiter=',', dtype=None, encoding=None)
path_test = './dataset/titanic/titanic_test_data.csv'
test_data = genfromtxt(path_test, delimiter=',', dtype=None, encoding=None)
y = data[1:, -1]  # label = survived
class_names = ["Died", "Survived"]
labeled_idx = np.where(y != '')[0]

y = np.array(y[labeled_idx])
y = y.astype(float).astype(int)

# Preprocessing dataset
print("\n\nPart (b): preprocessing the titanic dataset")
X, onehot_features = preprocess(data[1:, :-1], onehot_cols=[1, 5, 7, 8])
X = X[labeled_idx, :]
Z, _ = preprocess(test_data[1:, :], onehot_cols=[1, 5, 7, 8])
assert X.shape[1] == Z.shape[1]
features = list(data[0, :-1]) + onehot_features

print("Features:", features)
print("Train/test size:", X.shape, Z.shape)

print("\n\nPart 0: constant classifier")
print("Accuracy", 1 - np.sum(y) / y.size)

# Split into training and validation sets.
X_titanic = X
y_titanic = y
Z_titanic = Z
titanic_val, titanic_training = shuffle_partition(X_titanic, 150,999)

# Basic decision tree
print("\n\nPart (a-b): simplified decision tree")
dt_titanic = DecisionTree(max_depth=3, feature_labels=features)
dt_titanic.fit(X, y)
print("Predictions", dt_titanic.predict(Z)[:100])

# Random Forest
rf_titanic = RandomForest(random_states=6, max_depth=3, feature_labels=features)

print("\n\nPart (c): sklearn's decision tree")
clf = sklearn.tree.DecisionTreeClassifier(random_state=0, **params)
clf.fit(X, y)
evaluate(clf)
out = io.StringIO()

# You may want to install "gprof2dot"
sklearn.tree.export_graphviz(
    clf, out_file=out, feature_names=features, class_names=class_names)
graph = pydot.graph_from_dot_data(out.getvalue())
pydot.graph_from_dot_data(out.getvalue())[0].write_pdf("%s-tree.pdf" % dataset)



Part (b): preprocessing the titanic dataset
Features: ['pclass', 'sex', 'age', 'sibsp', 'parch', 'ticket', 'fare', 'cabin', 'embarked', 'male', 'female', 'S', 'C', 'Q']
Train/test size: (999, 14) (310, 14)


Part 0: constant classifier
Accuracy 0.6166166166166166


Part (a-b): simplified decision tree
Predictions [1. 0. 0. 0. 1. 1. 1. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 1. 0. 1. 1. 0.
 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 1.
 1. 1. 0. 0. 1. 0. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.
 1. 1. 1. 0. 1. 0. 0. 0. 1. 1. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 1. 0.]


Part (c): sklearn's decision tree
Cross validation [0.795      0.825      0.805      0.755      0.74371859]


In [390]:
# Spam dataset

dataset = "spam"
params = {
    "max_depth": 5,
    # "random_state": 6,
    "min_samples_leaf": 10,
}
N = 100


features = [
    "pain", "private", "bank", "money", "drug", "spam", "prescription", "creative",
    "height", "featured", "differ", "width", "other", "energy", "business", "message",
    "volumes", "revision", "path", "meter", "memo", "planning", "pleased", "record", "out",
    "semicolon", "dollar", "sharp", "exclamation", "parenthesis", "square_bracket",
    "ampersand"
]
assert len(features) == 32

# Load spam data
path_train = './dataset/spam/spam_data.mat'
data = scipy.io.loadmat(path_train)
X = data['training_data']
y = np.squeeze(data['training_labels'])
Z = data['test_data']
class_names = ["Ham", "Spam"]

   
print("Features:", features)
print("Train/test size:", X.shape, Z.shape)

print("\n\nPart 0: constant classifier")
print("Accuracy", 1 - np.sum(y) / y.size)

# Split into training and validation sets.
X_spam = X
y_spam = y
Z_spam = Z
spam_val, spam_training = shuffle_partition(X_titanic, 900,5280)

# Basic decision tree
print("\n\nPart (a-b): simplified decision tree")
dt_spam = DecisionTree(max_depth=3, feature_labels=features)
dt_spam.fit(X, y)
print("Predictions", dt_spam.predict(Z)[:100])

# Random forests
rf_spam = RandomForest(random_states=6, max_depth=5, feature_labels=features)

print("\n\nPart (c): sklearn's decision tree")
clf = sklearn.tree.DecisionTreeClassifier(random_state=0, **params)
clf.fit(X, y)
evaluate(clf)
out = io.StringIO()

# You may want to install "gprof2dot"
sklearn.tree.export_graphviz(
    clf, out_file=out, feature_names=features, class_names=class_names)
graph = pydot.graph_from_dot_data(out.getvalue())
pydot.graph_from_dot_data(out.getvalue())[0].write_pdf("%s-tree.pdf" % dataset)

X_spam = X
y_spam = y
Z_spam = Z

Features: ['pain', 'private', 'bank', 'money', 'drug', 'spam', 'prescription', 'creative', 'height', 'featured', 'differ', 'width', 'other', 'energy', 'business', 'message', 'volumes', 'revision', 'path', 'meter', 'memo', 'planning', 'pleased', 'record', 'out', 'semicolon', 'dollar', 'sharp', 'exclamation', 'parenthesis', 'square_bracket', 'ampersand']
Train/test size: (5280, 32) (5749, 32)


Part 0: constant classifier
Accuracy 0.7329545454545454


Part (a-b): simplified decision tree
Predictions [0. 0. 0. 0. 1. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 1. 0. 1. 0. 1. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0.
 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 1.
 0. 0. 1. 0.]


Part (c): sklearn's decision tree
Cross validation [0.80681818 0.81912879 0.82481061 0.80965909 0.81628788]


In [391]:
# 4.4 Evaluation

# Train decision tree on titanic training data and predict on validation data.
dt_titanic = DecisionTree(max_depth=3, feature_labels=features)
dt_titanic.fit(X_titanic[titanic_training], y[titanic_training])
dt_titanic_predictions = dt_titanic.predict(X_titanic[titanic_val])
print(accuracy(dt_titanic_predictions, y_titanic[titanic_val]))

# Train random forest on titanic training data and predict on validation data.
rf_titanic = RandomForest(random_states=6, max_depth=3, feature_labels=features)
rf_titanic_predictions = rf_titanic.fit_and_predict(X_titanic[titanic_training], y_titanic[titanic_training], X_titanic[titanic_val])
print(accuracy(rf_titanic_predictions, y_titanic[titanic_val]))

# Train decision tree on spam training data and predict on validation data.
dt_spam = DecisionTree(max_depth=3, feature_labels=features)
dt_spam.fit(X_spam[spam_training], y_spam[spam_training])
dt_spam_predictions = dt_spam.predict(X_spam[spam_val])
print(accuracy(dt_spam_predictions, y_spam[spam_val]))

# Train random forest on spam training data and predict on validation data.
rf_spam = RandomForest(random_states=6, max_depth=5, feature_labels=features)
rf_spam_predictions = rf_spam.fit_and_predict(X_spam[spam_training], y_spam[spam_training], X_spam[spam_val])
print(accuracy(rf_spam_predictions, y_spam[spam_val]))

0.5533333333333333
0.7466666666666667
0.8077777777777778
0.8322222222222222


In [392]:
import csv

header = ['Id', 'Category']
data = []
def create_file(prediction_file, file_name):
    for i in range(len(prediction_file)):
        data.append([i+1, int(prediction_file[i])])
    with open(file_name, 'w', newline = '') as f:
        writer = csv.writer(f, delimiter=',')
        writer.writerow(header)
        writer.writerows(data)
    f.close()

In [394]:
# Test predictions for Titanic
test_predictions = rf_titanic.fit_and_predict(X_titanic, y_titanic, Z_titanic)
create_file(test_predictions, "titanic_predictions.csv")

In [395]:
import csv

header = ['Id', 'Category']
data = []
def create_file(prediction_file, file_name):
    for i in range(len(prediction_file)):
        data.append([i+1, int(prediction_file[i])])
    with open(file_name, 'w', newline = '') as f:
        writer = csv.writer(f, delimiter=',')
        writer.writerow(header)
        writer.writerows(data)
    f.close()

In [397]:
# Test predictions for SPAM

test_predictions = rf_spam.fit_and_predict(X_spam, y_spam, Z_spam)
create_file(test_predictions, "spam_predictions.csv")